# Orbital Orchestra of the Machine

## Using a Neural Network to Observe the N-Body Problem


### !!! FOREWARNING, READ BEFORE YOU RUN !!!
I have not tested the effects of running all three scripts at once using Jupyter, my intent of this document is to be documentation, not the main scripts to run. I highly recommend downloading the scripts, adjusting the filepath to your own selected path in each script, and run them separately. To properly execute script 2 you may need decent hardware, a way to cool your hardware, and plenty of time for running a few thousand training sessions. I hope the exclamations caught your attention. Ignoring my warning is your choice, if you choose to do so you agree to whatever responsibility may come about. 

### Introduction
Let's start with how this project originally came to be. Astrophysics is the topic that interests me the most when it comes to physics. When it comes to the cosmos, I can't help but marvel at the size of literally everything, and how everything just seamlessly works together. The way we as people challenge this final frontier of outer space with what we can while only being able to observe just so much of it. I tend to dream of being involved in future missions in some capacity, so much so that I'd think of imaginary flight systems. Thoughts of GPS systems came to mind as well, I learned that they use trilateration to calculate a device's at regular intervals down here on Earth. That thought train ran to how could we implement a GPS system way out there? Rather, how do we keep track of various space objects as they move in various directions along our flight paths? This thought train is what initially led me to choose a project relating to chapter 4 of "Computational Modeling and Visualization of Physical Systems", the text written by and provided in Professor Wang's Computational Physics module, where the chapter looks at planetary motion and n-body simulations. 

However, this past summer hit me with a different sort of reality. As I worked on my job search for post-graduation, I received some feedback for my applications. I would not be considered for my lack of AI and machine learning skills. My plan was then to use this project to gain that skill, yet I still wanted to work on an n-body simulation. Hence this topic came to be. I would merge the two and use machine learning to predict orbits, essentially letting me work on that simulation and gain that skill. 

We begin the three-body simulation of the Sun, the Earth, and Jupiter below, a modification of the Earth-Sun system featured in program 4.1 and figure 4.1 of the text. 
Original modifications included a Python 3.x conversion, updating the format and modules used, and the addition of the rest of the planets in our solar system. For the sake of this project and future model training, all planets except Earth and Jupiter were removed, but their data is stored along with all the scripts made in my github. Since the original modified program contained all planets, please assume these scripts were made with their potential reinclusion in mind. They were, with the process only requiring a pasting from the full planet data library into the 'bodies' dictionary for the desired celestial object. 

### Section 1: N-body Simulation (AKA Solar System Sim)

In [ ]:
import numpy as np
from vpython import vector, mag, sphere, canvas, rate, textures, color  #Replaces visual module, used to generate the scene and for vector functions
import csv  #Used for saving data in a spreadsheet
import os   #Used for file system functions


We'll import numpy for numerical functions such as including $\pi$, and select functions from vpython for work in 3D space.
'vector' allows the use of vectors and 'mag' allows the collection of the magnitude of those vectors, which is helpful for tasks like obtaining distances between bodies. 'sphere', 'canvas', 'textures', 'color' were needed to create the 3D visual scene, the objects, and give them color or, solely in Earth's case, a texture. 'rate' allows the controlling of simulation speed. 

csv was imported so that I could export the data collected on the planetary motion into a spreadsheet that would be used to train my model. OS was imported to use a certain directory throughout all scripts, have files written or data saved or pulled, save the model, retrieve the model. All done through the same directory. 

In [ ]:
bodies = [
   {
        'name': 'Sun',
        'mass': 1.0,  #In solar masses
        'pos': vector(0, 0, 0),  #Initial position, in AU
        'vel': vector(0, 0, 0),  #Initial velocity, in AU/yr
        'radius': 0.2,           #Size of sphere, visual purposes only
        'color': color.yellow,
        'texture': None,         #Because I wanted Earth to have the Earth texture, all bodies must have this line set to 'none' for how the bodies are called in later. Tedious but aides in simplicity later.  
    },
    {
        'name': 'Earth',
        'mass': 3.0e-6,  #In Solar masses
        'pos': vector(1.0, 0, 0),  # Distance ~1 AU from the Sun
        'vel': vector(0, 0, -6.179),  # Pulled from table in cpms-ch04
        'radius': 0.1,             #Visual purposes only
        'color': color.white,      #For blank sphere so texture can be the sphere's "color"
        'texture': textures.earth, #home
    },
    {
        'name': 'Jupiter',   #Chosen for it's large mass. Helps cause of perturbation on the Sun
        'mass': 9.5e-4,      #In solar masses. All masses are pulled from the table in cpms-ch04
        'pos': vector(5.2, 0, 0),  # Distance ~5.2 AU from the Sun
        'vel': vector(0, 0, -2.624),  # Pulled from table in cpms-ch04
        'radius': 0.15,
        'color': color.orange,
        'texture': None,
    },
]

Above we create the list of dictionaries for each celestial body. For organization we start with the centermost object and head outward as we descend In the full list, it is the Sun, Mercury, so on up to Pluto. The structure of the dictionary is integral to the scripts. It's the format used in all scripts, and the format that the model must recognize. We start with the 'name' of the body, that will also be used to automate labeling of columns in data collection. Then we have the 'mass' of the objects in terms of Solar masses. These get recalled when working with the equations of motion later on. Moving on to initial positions and velocities, both metrics get stored in a 'vector' vector to be used later on, and their values are pulled from table 4.1, "Properties of Kepler's orbits of the Planets" from the text. 'Radius' for each body is an unrealistic number used only to visualize the objects when the scene renders. However, it is also in AU for those who'd wish to know. 'Color' is the color of the sphere when the scene loads, we use white for Earth so that we may view the texture I was picky about in its full glory instead of painting a deep blue over it as I did in my first several runs of the prediction simulation.  

In [ ]:
def compute_accelerations(bodies):
    
    # Initialize a list of accelerations with zero vectors for all mentioned bodies. (Can include others)
    accels = [vector(0,0,0) for _ in range(len(bodies))]
    
    # For each pair of bodies i, j, compute gravitational acceleration contribution
    for i in range(len(bodies)):
        for j in range(len(bodies)):
            if i != j: #loop works as long as the two bodies referenced aren't the same. 
                # Relative position vector from body j to i
                rij = bodies[i]['pos'] - bodies[j]['pos']
                dist = mag(rij)
                # Acceleration contribution from body j on i, added to list in location for specified body.
                accels[i] += -G * bodies[j]['mass'] * rij / (dist**3)
    return accels

We create a function used to calculate the gravitational acceleration experienced by each body we've specified above by every other body using two 'for' loops and an 'if.. does not equal' loop to verify that we do not match two like bodies (e.g. Earth pulling on Earth, prevents non-physical results and division by zero) yet still captures the interaction of every body on each other. The outer loop goes over each body as the target body, and the inner loop goes over each body as the source of gravitational force exerted on to the target body. Only if body i does not equal body j does the rest of the function run. 

Once i not equalling j is verified, the function computes the vector pointing from body j to body i and stores the info into 'rij', whose magnitude is computed the next line down. Finally, the acceleration of that body is then added into accels[i] which acts as the total acceleration for that body. 
Plainly, we need the accelerations as they dictate the changes in velocity as the simulation progresses through time. We use the idea that 

$$a_i = \frac{F_{ij}}{m_i} = -G \frac{m_j}{r^3_{ij}}\mathbf{r}_{ij}$$

Which is aforementioned acceleration added to accels[i]
The function finally returns the acceleration for use later on. 

In [ ]:
def save_data_incremental(t, bodies, filename="C:\\Users\\Manny Admin\\Desktop\\New Data\\Simulation Pull\\simulation_data.csv"):

    if not os.path.exists(filename):
        # Write the header if the file doesn't exist
        with open(filename, mode='w', newline='') as file:
            writer = csv.writer(file)
            headers = ["time"]
            for body in bodies:
                #Creates the rest of the header for the .csv file (e.g Sun_x, Mercury_vz) for all bodies in order listed above.
                headers.extend([f"{body['name']}_x", f"{body['name']}_y", f"{body['name']}_z",
                                f"{body['name']}_vx", f"{body['name']}_vy", f"{body['name']}_vz"])
            writer.writerow(headers)

    # Append the data row to the csv file
    with open(filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        row = [t]
        for body in bodies:
            row.extend([body['pos'].x, body['pos'].y, body['pos'].z,
                        body['vel'].x, body['vel'].y, body['vel'].z])
        writer.writerow(row)

This function was designed to save the metric data for each celestial body at every timestep of the simulation and added for the PHY 480 project. It's incremental and not a batch save at the end because initially my original modified sim ran indefinitely and I was working off of that. This method still works for what I needed, and remains unchanged even after introducing an end time to cease simulating. The variables for this function are the time step involved, the 'bodies' list for the bodies' names, position, and velocity data at time. I have thought about adding acceleration data, I think the model might benefit from the extra data, but for the sake of time that will be a future adjustment. 

The upper loop creates the header in the event that the file doesn't exist in my specified directory. It first checks with the if not condition to see if the file exists, it does now so this is skipped, but if it didn't we'd move to the next few lines. The open function opens the file in 'W'rite mode after creating it. CSV.writer is a writer object that will convert the inputted data into strings within the file. Finally, the headers are created. The first one is for 'time', and the rest are the position in x, y, z, format then the velocities in vx, vy, vz format for each celestial body in 'bodies'. The second half, lower loop, of the function is similar to the first in structure, with a lack of file check. The second half also does not create headers, it appends the data from the simulation into the columns generated. 

In [1]:
def simulate():
    
    #Run the N-body simulation using a Leapfrog integrator and visualize it.
    
    # Create the scene
    scene = canvas(title="N-Body Simulation", width=800, height=600)
    scene.autoscale = True
    
    # Create spheres for each body for visualization
    for b in bodies:
        b['obj'] = sphere(
            pos=b['pos'],
            radius=b['radius'],
            color=b['color'],
            texture=b['texture'],
            make_trail=True,
            retain=5000  # Keep a long trail
        )
    
    # Time parameters
    t = 0.0        # Start time
    h = 0.001       # Timestep in years
    simulation_duration = 10.0  # Total simulation duration in years
    data = []       # Storage for simulation data
    
    # Initial accelerations
    accels = compute_accelerations(bodies)
    
    # Compute half-step velocities for leapfrog integrator
    for i, b in enumerate(bodies):
        # v_half = v + 0.5 * a * h
        b['v_half'] = b['vel'] + 0.5 * accels[i] * h

The goal of this function is as commented above. Run the simulation using a Leapfrog Integrator and visualize it. Though now we have the added caveat of saving our data. The function starts by setting the scene in a literal sense. The canvas function initializes a 3D rendered space that visualizes the simulation. Next, spheres are created using the list of bodies, and the dictionaries within for all the necessary visual features like position, color, etc. The spheres will also have a trail to help visualize the object's path. Next, we define the initial time, and timestep in years. Every timestep is 0.365 Earth days. or 0.001 Earth years. The simulation lasts for 10 years to generate a lengthy data set with repeated earth orbits. I believed it would help in model training compared to the initial one year I tried before, and the model displayed some attempt at maintaining the orbits. The storage list for the data is also defined. 
Next, we gather the initial accelerations using the compute_accelerations function defined earlier. These will be used in the leapfrog integrator.  

Finally we compute the half-step velocities of each of the celestial bodies to prep for leapfrog integration, the for loop here goes over every body in the list, providing both the index 'i' and dictionary 'b'. The half step calculation is given by ... tbc



In [2]:
    
    # Main simulation loop
    while t < simulation_duration:
        rate(200)  # Limit to 200 frames per second
        
        # Save current timestep data incrementally
        save_data_incremental(t, bodies)
        
        # Update positions of all bodies using v_half
        for i, b in enumerate(bodies):
            # new_r = r + v_half * h
            b['pos'] = b['pos'] + b['v_half'] * h
        
        # Compute new accelerations after moving bodies
        accels = compute_accelerations(bodies)
        
        # Update velocities for all bodies
        for i, b in enumerate(bodies):
            # new_v_half = v_half + a * h
            b['v_half'] = b['v_half'] + accels[i] * h
            
            # Update the sphere positions in the scene
            b['obj'].pos = b['pos']
        
        t += h

NameError: name 't' is not defined

The rest of the simulation function resides in this while loop. It uses our save_data_incremental() funtion defined earlier to save data at that time step, then utilizes the other equations of leapfrog integration shown in the previous block of text to compute the necessary information for the next time step, all while updating the proper variables. 
The simulation lasts up until the time duration, through the run time and after saving data the program uses another loop ... tbc

In [ ]:
simulate()

### Section 2, Second Script: PINN Model Creation and Training

In [ ]:
import torch  # Import PyTorch library for building and training the neural network.
from torch.utils.data import Dataset, DataLoader  # Utilities for dataset handling and batching.
import pandas as pd  # Pandas for data manipulation and analysis.
import numpy as np  
import os 

These are the modules and functions I've called for my script. PyTorch or 'torch' is the center of my script, its a framework for developing neural networks. Dataset and DataLoader from torch.utils.data are necessary for properly batching the data for my model. Pandas also assists in data preprocessing as its used for file manipulation. 

In [ ]:
# Dataset for Solar System Simulation Data 
class SolarSystemDataset(Dataset):

This inherits from torch.utils.data.dataset class, and is used so that we can easily work with PyTorch's data loading utilities, allowing for efficient data batching, shuffling and parallel processing.

Data batching is the process of grouping multiple samples in a single "batch" before "feeding" to a neural network for training. Modern CPUs and GPUs are optimized for parallel computations, batching allows the use of vectorized operations, which help speed up processing over individual samples. 

Data shuffling is the random rearrangement of data samples before a training epoch occurs.

Parallel processing is the use of multiple worker processes or threads to load and preprocess data at once.  

In [ ]:
    #Initialize the dataset by loading data from a CSV file.
    #csv_file: Path to the CSV file containing simulation data.
    def __init__(self, csv_file):

The role of this function is to initialize the the attributes of the current class instance and conduct and setup processes before the object is used. This is the basis for the "Constructor" method for Python classes. 

'self' refers to the instance of the class itself, and allows access to the attributes of the class. 

Any other argument in this line equates to the required inputs to initialize the object, in this instance (pun) its the csv_file made in the previous script. 

In [ ]:
        # Load the data from the CSV file into the Dataframe using Panads.
        self.data = pd.read_csv(csv_file)

This line uses pandas 'pd' to load the data from the csv_file using the 'read_csv' function and placed into the dataframe as instance attribute self.data

In [ ]:

    def __len__(self):
       
        # Return the total number of samples in the dataset.
        
        # Using length-1 because I need a "next state" for each sample.
        return len(self.data) - 1

The '_ len__' method returns the total number of samples in a dataset. I used it as 'len(self.data) - 1' because I needed to verify that each sample in the dataframe has a 'current state' and 'next state' pairing. 

In [1]:
    def __getitem__(self, idx):
        
        # Current state (all columns except time in column 0).
        current_state = self.data.iloc[idx, 1:].values.astype(np.float32)
        
        # Next state (the next row in the dataset).
        next_state = self.data.iloc[idx + 1, 1:].values.astype(np.float32)

        return torch.tensor(current_state), torch.tensor(next_state)


The purpose of this getitem method is to capture instances of my dataset class and index '[]'  them as individual samples based on an index. This will allow efficient data access and manipulation during training. The first line takes the data, column 1,  at a specified location of that indexed instance, row 'idx', and stores it in that variable, current_state, as a lengthy float to capture the minor variations between states. Next_state is essentially the exact same as current_state, with a tweak to capture the data for the next instance. 

Finally, the numpy arrays are converted and returned as torch tensors. These tensors are crucial when working with PyTorch structures as they're optimized for GPU acceleration and automatic differentiation. 

Next, we'll define the PINN class. 

In [ ]:
# Define the Neural Network 
class PINN(torch.nn.Module):
    def __init__(self, input_size, output_size, hidden_layers=3, hidden_units=128):
                
        super(PINN, self).__init__()


Here, we define the Physics Informed Neural Network, the PINN class. PINNs integrate physical laws and equations into the training process, which should allow the model to get a better grasp of orbital mechanics in my case. 

'torch.nn.module' serves as the base class for all neural networks, and is essentially what kept me from starting entirely from scratch when switching from an LSTM model to a GNN, now to a PINN. By inheriting from this class, the model gains essential functions like parameter management, the functionality to move to different devices (CPU/GPU), and the forward pass function that I use later on. 

The constructor method for this class is defined, this time with many more parameters. 'Input_size' refers to the dimensions of the data going into the model for training, in this case its position in 3D space (x, y, z), plus the velocities in 3D space (vx, vy, vz) times the number of bodies we want to predict (Sun, Earth, Jupiter, so 3). In our system it is 18. 

(Important note, this is the only non-automatic aspect to adding more bodies, that model's parameters. However it is basic math and just changing the value)

Output_size mirrors input_size as we want to guess the next state, so also 18. 

hidden units, the number of neurons hidden in each layer, retains its default value of 128. This number affects the model's learning capacity and computational efficiency, and because it works it remains unchanged. 

The 'super' function initializes the parent torch.nn.module class, and is crucial for setting up parameter tracking and device placement from PyTorch modules.

Next, the layers. 

In [ ]:
        # Layers of the model
        layers = []

        # Input layer
        layers.append(torch.nn.Linear(input_size, hidden_units))
        layers.append(torch.nn.ReLU())

        # Hidden layers
        for _ in range(hidden_layers):
            layers.append(torch.nn.Linear(hidden_units, hidden_units))
            layers.append(torch.nn.ReLU())

        # Output layer
        layers.append(torch.nn.Linear(hidden_units, output_size))

        self.model = torch.nn.Sequential(*layers)

First we initialize an empty list to store the layers of the neural network using 'layers = []'

The torch.nn.Linear function creates a linear, fully connected layer that in this case maps input features to the first hidden layer, this is appended to layers. The linear transformation of this process looks like 

$$y = xA^T + b$$

with $A$ being the weight matrix and $b$ the bias vector. 

torch.nn.ReLU() adds a Rectified Linear Unit activation function that introduces nonlinearity into the model, allowing the learning of complex patterns and odd trajectories. This also gets appended into layers. 

The for loop iterates as many times as there are hidden layers. The contents of the loop connects one hidden layer to the next, creating a map of connected hidden layers based off the number of hidden units in each layer. In this case every hidden layer has the same amount of hidden units.

This along with the applied ReLU function get appended to layers. 

Next, the final fully connected layer connecting the last hidden layer to the output layer is created, and appended. 

torch.nn.Sequential chains the list of layers into a single sequential model. *layers unpacks the list passing each layer as a separate argument, resulting in the newly built neural network model, self.model, as a sequential container that applies each layer when data gets passed through.


In [ ]:
    def forward(self, x):
        
        #Forward pass through the network.
        # x: Input tensor (current state).
        # return: Output tensor (predicted next state).
        
        return self.model(x)


Here we define the forward pass of the neural network. It returns our model as a function, self.model(x), that passes input 'x' through its sequentially defined layers. 

In [2]:
def compute_loss(predictions, targets, positions, velocities, masses, G=4*(np.pi**2), Δt=0.001):

IndentationError: expected an indented block (2718886.py, line 2)

This function is integral to the design of my PINN model setup, here we add in the equations of motion that make up the physics aspect to the model's training regimen, allowing it to hopefully get a glimmer of understanding of orbital dynamics. The arguments are as follows

'predictions' is a PyTorch tensor, its the generated output given to us by the model and provides the predicted positions and velocities at the next time step. 

'targets' is also a PyTorch tensor that contains the actual next state of the system based on the simulation we ran in the previous script. 

'positions' another involved tensor, it contains the spatial data at a given time step.

'velocities' similar to 'positions', a tensor that contains the velocities at a given timestep.

'masses' a tensor of constants, its in tensor format to simplify training processes. 

'G' is the gravitational constant set to 4$\pi^2$ to remain consistent with the previous script. 

'Δt' is the time step according to 'h' in the previous script. Consistency is key to making this project work. 

In [ ]:

    data_loss = torch.nn.functional.mse_loss(predictions, targets)

torch.nn.functional.mse_loss computes the Mean Squared Error between the predictions and target data, this encourages the model to produce outputs closer to the target data.

In [ ]:

    batch_size, total_features = predictions.shape
    num_bodies = total_features // 6  # each body: 3 pos + 3 vel
    pos_dim = num_bodies * 3
    vel_dim = num_bodies * 3

the batch size refers to the number of samples in the current batch, and the total_features is the total number of features per sample. Both make up the shape of the current prediction.
We get num_bodies by dividing the number of total_features (18) by 6 (3 for position coordinates, 3 for velocity components.) 
pos_dim and vel_dim refer to the number of dimensions (features) each metric has (3 each). 

In [ ]:

    # Extract predicted positions and velocities
    predicted_positions = predictions[:, :pos_dim]
    predicted_velocities = predictions[:, pos_dim:pos_dim+vel_dim]

predicted_positions selects the rows in the batch and columns from the start up to 'pos_dim', and represents the predicted positions of all celestial bodies. 

predicted_velocities selects the rows in the batch and columns from 'pos_dim' up until 'pos_dim + vel_dim', and represents the predicted velocities for all celestial bodies. 

In [ ]:

    predicted_accelerations = (predicted_velocities - velocities) / Δt

This line computes the change in velocity over time step Δt, and predicted_accelerations represent the model's estimated changes in velocites based off the predicted velocities. This assumes $$a = \frac{Δv}{Δt} $$

In [ ]:
    physics_loss = 0.0
    for i in range(num_bodies):
        force_residual = torch.zeros_like(predicted_positions[:, i*3:(i+1)*3])
        for j in range(num_bodies):
            if i != j:
                r_ij = positions[:, i*3:(i+1)*3] - positions[:, j*3:(j+1)*3]
                # Clamping the distance to a minimum of 1 to avoid huge forces exploding the training loss. (Had happened.)
                dist = torch.clamp(torch.norm(r_ij, dim=1, keepdim=True), 1)
                force = -G * masses[j] * r_ij / (dist**3)
             
                force_residual += force

'physics_loss = 0.0' initializes the physics loss variable to soon hold all physics based loss across all bodies in a batch. 

The for loop iterates over each body to capture the loss data.
force_residual initializes with a tensor of zeros with the same shape as the position vector of the ith body with the goal of accumulating the gravitational forces exerted on the ith body by the rest of the bodies. Seems similar to something we did earlier because it is, however this time we use torch tensors for simplified communication with the model. 

the for j loop iterates all bodies to capture the gravitational force of all bodies acting on the ith body, and prevent ith on ith interactions using the != condition check. 

As before, r_ij represents the vector displacement from the jth body to the ith body. 

torch.norm(r_ij, dim = 1, keepdim = True) calculates the Euclidean distance between the ith and jth bodies, and torch.clamp(..., 1) is used in this case to restrict the minimum distance to 1 AU to prevent the bodies from approaching the sun too closely and exploding the loss values. For the inclusion of Venus and Mercury this should be changed to their respective distances as minimums, but for simplicity we'll keep it at Earth's 1 AU. We then use Newton's law of gravitation to calculate the force and accumulate the result into force_residual. 

Any other physical interactions should be included in this section. 

In [ ]:

        # Compute physics loss for body i
        physics_loss += torch.mean((predicted_accelerations[:, i*3:(i+1)*3] - force_residual)**2)

Working from in heading out, 'predicted_accelerations[:, i*3:(i+1)*3] - force_residual)**2' represents the squared difference between the model's predicted acceleration and teh physically computed acceleration based on gravitational forces. When compared to actual astronomical data there should be differences based on the interactions not included abnove, but since this section and the simulation primarily only focus on the gravitational interaction, it should work out for what I need. 

torch.mean computes the mean squared error for all samples in the batch for the ith body's acceleration. 

This gets accumulated into physics_loss. 

In [ ]:
    physics_loss = physics_loss / num_bodies

Normalizes the physics loss across all bodies. 

In [ ]:
    return data_loss, physics_loss

In [ ]:
data_loss is the MSE between the model's predictions and 

In [ ]:
if __name__ == "__main__":
    # Set working directory
    os.chdir("C:\\Users\\Manny Admin\\Desktop\\New Data\\Simulation Pull")

In [ ]:
    # CSV file path
    csv_file = "simulation_data.csv"

In [ ]:
    # Initialize dataset
    dataset = SolarSystemDataset(csv_file)

In [ ]:
Place holder

In [ ]:
    # Split dataset into training (80%) and validation (20%)
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

In [ ]:
Place holder

In [ ]:
    # Data loaders
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
Place holder

In [ ]:
    # Determine input/output size
    sample_input, sample_target = dataset[0]
    input_size = len(sample_input)
    output_size = len(sample_target)

In [ ]:
Place holder

In [ ]:
    # Initialize model
    model = PINN(input_size, output_size)
    print(model)

In [ ]:
Place holder

In [ ]:
    # Optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-5) #Adjust Learning Rate Here

In [ ]:
Place holder

In [ ]:
    # Training parameters
    epochs = 200
    masses = torch.tensor([1.0, 3.004e-6, 9.551e-4], dtype=torch.float32)

In [ ]:
Place holder

In [ ]:
    # Physics introduction parameters
    start_physics_epoch = 10
    max_physics_weight = 1e-4 #Adjust weight of physics laws on training here. 0 physics creates the BlackBox model. 
    physics_weight = 0.0 # Initial physics weight, allows the model to get a grasp of the data and adjust smoother.


In [ ]:
Place holder

In [ ]:
    for epoch in range(epochs):
        model.train()
        train_loss_sum = 0.0

        # After start_physics_epoch, gradually increase physics weight
        if epoch > start_physics_epoch:
            physics_weight = (epoch - start_physics_epoch) * (max_physics_weight / 10.0)
            physics_weight = min(physics_weight, max_physics_weight)
        else:
            physics_weight = 0.0

        for batch_idx, batch in enumerate(train_loader):
            current_state, next_state = batch

            num_bodies = input_size // 6
            pos_dim = num_bodies * 3
            vel_dim = num_bodies * 3

            positions = current_state[:, :pos_dim]
            velocities = current_state[:, pos_dim:pos_dim+vel_dim]

            predictions = model(current_state)

            data_loss, physics_loss = compute_loss(predictions, next_state, positions, velocities, masses)

            total_loss = data_loss + physics_weight * physics_loss

            optimizer.zero_grad()
            total_loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            train_loss_sum += total_loss.item()

        avg_train_loss = train_loss_sum / len(train_loader)

        # Validation
        model.eval()
        val_loss_sum = 0.0
        with torch.no_grad():
            for batch in val_loader:
                current_state, next_state = batch
                positions = current_state[:, :pos_dim]
                velocities = current_state[:, pos_dim:pos_dim+vel_dim]
                predictions = model(current_state)
                data_loss_val, physics_loss_val = compute_loss(predictions, next_state, positions, velocities, masses)
                val_total_loss = data_loss_val + physics_weight * physics_loss_val
                val_loss_sum += val_total_loss.item()

        avg_val_loss = val_loss_sum / len(val_loader)
        
        #print function to visualize progress. (Fun tidbit. Losses once began in the octillions because I added a safeguard against division by 0. All I had to do was let the model know those columns were supposed to be 0 entirely)
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Physics Weight: {physics_weight:.2e}")


In [ ]:
Place holder

In [ ]:

    # Save the trained model to my usual directory
    model_save_path = os.path.join("C:\\Users\\Manny Admin\\Desktop\\New Data\\Simulation Pull", "PINN_model.pth")
    torch.save(model.state_dict(), model_save_path)
    print(f"Model saved to {model_save_path}")

In [ ]:
Place holder

In [ ]:
Place holder

In [ ]:
Place holder

In [ ]:
Place holder

### Section 3, Third Script: Prediction Simulation

Place holder

In [1]:
import numpy as np
from vpython import vector, sphere, canvas, rate, textures, color
import torch
import os


<IPython.core.display.Javascript object>

ModuleNotFoundError: No module named 'torch'

In [ ]:
Place holder

In [ ]:
class PINN(torch.nn.Module):
    def __init__(self, input_size, output_size, hidden_layers=3, hidden_units=128):
        
        #Initialize the PINN model
        #input_size: Number of input features (positions and velocities).
        #output_size: Number of output features (positions and velocities).
        #hidden_layers: Number of hidden layers in the neural network.
        #hidden_units: Number of neurons per hidden layer.
        
        super(PINN, self).__init__()

        # Layers of the model
        layers = []

        # Input layer
        layers.append(torch.nn.Linear(input_size, hidden_units))
        layers.append(torch.nn.ReLU())

        # Hidden layers
        for _ in range(hidden_layers):
            layers.append(torch.nn.Linear(hidden_units, hidden_units))
            layers.append(torch.nn.ReLU())

        # Output layer
        layers.append(torch.nn.Linear(hidden_units, output_size))

        self.model = torch.nn.Sequential(*layers)

In [ ]:
Place holder

In [ ]:

    def forward(self, x):
        
        #Forward pass through the network.
        #Input tensor (current state).
        #Output tensor (predicted next state).
        
        return self.model(x)

In [ ]:
Place holder

In [ ]:
bodies = [   #Can add others if others are available. Match with sim info.
    {
        'name': 'Sun',
        'mass': 1.0,
        'pos': vector(0, 0, 0),
        'vel': vector(0, 0, 0),
        'radius': 0.2,
        'color': color.yellow,
        'texture': None,
    },
    {
        'name': 'Earth',
        'mass': 3.0e-6,
        'pos': vector(1.0, 0, 0),  
        'vel': vector(0, 0, -6.179),  
        'radius': 0.1,
        'color': color.white,
        'texture': textures.earth,
    },
    {
        'name': 'Jupiter',
        'mass': 9.5e-4,
        'pos': vector(5.2, 0, 0),  
        'vel': vector(0, 0, -2.624),  
        'radius': 0.15,
        'color': color.orange,
        'texture': None,
    },
]

In [ ]:
Place holder

In [ ]:
def load_model(model_path):
    
    # Load the pre-trained PINN model from a .pth file.
    
    # Define the model architecture (ensure this matches the training script)
    input_size = 18  # Update this to match your dataset's input size
    output_size = 18  # Update this to match your dataset's output size
    model = PINN(input_size=input_size, output_size=output_size, hidden_layers=3, hidden_units=128)

    # Load the state dictionary
    state_dict = torch.load(model_path, map_location=torch.device('cpu'))
    model.load_state_dict(state_dict)

    # Set the model to evaluation mode
    model.eval()
    return model

In [ ]:
Place holder

In [ ]:
def get_state_vector(bodies):
   
    # Construct the state vector (positions and velocities) from the list of bodies.
    # Order: [x,y,z,vx,vy,vz] for each body in the order they are listed in 'bodies'.
    
    state = []
    for b in bodies:
        state.append(b['pos'].x)
        state.append(b['pos'].y)
        state.append(b['pos'].z)
        state.append(b['vel'].x)
        state.append(b['vel'].y)
        state.append(b['vel'].z)
    return torch.tensor(state, dtype=torch.float32).unsqueeze(0)  # shape [1, features]


In [ ]:
Place holder

In [ ]:
def update_bodies_from_state(bodies, state):
   
    # Update the positions and velocities of the bodies from the given state vector.
    # state is a torch tensor of shape [1, total_features], same order as get_state_vector.
    
    state = state.squeeze(0).detach().numpy()  # convert to numpy, shape [features]
    num_bodies = len(bodies)
    for i, b in enumerate(bodies):
        idx = i * 6
        b['pos'].x = state[idx]
        b['pos'].y = state[idx+1]
        b['pos'].z = state[idx+2]
        b['vel'].x = state[idx+3]
        b['vel'].y = state[idx+4]
        b['vel'].z = state[idx+5]


In [ ]:
Place holder

In [ ]:
def simulate():
    # Set the working directory to where model and data are located
    os.chdir("C:\\Users\\Manny Admin\\Desktop\\New Data\\Simulation Pull")


In [ ]:
Place holder

In [ ]:

    # Load the trained model
    model_path = "PINN_model.pth"
    model = load_model(model_path)

In [ ]:
Place holder

In [ ]:

    # Create the scene
    scene = canvas(title="N-Body Simulation (Model Predicted)", width=800, height=600)
    scene.autoscale = True

In [ ]:
Place holder

In [ ]:
    # Create spheres for each body for visualization
    for b in bodies:
        b['obj'] = sphere(
            pos=b['pos'],
            radius=b['radius'],
            color=b['color'],
            texture=b['texture'],
            make_trail=True,
            retain=5000
        )

In [ ]:
Place holder

In [ ]:
    
    # Time parameters
    t = 0.0        # Start time
    h = 0.001       # Timestep in years
    simulation_duration = 1.0  # total simulation time in years


In [ ]:
Place holder

In [ ]:

    while t < simulation_duration:
        rate(200)  # 200 frames per second

        # Get current state
        current_state = get_state_vector(bodies)

        # Use the model to predict the next state
        # The model should output the next positions and velocities after 1 timestep
        predictions = model(current_state)

        # Update bodies from the predicted next state
        update_bodies_from_state(bodies, predictions)

        # Update the sphere positions in the scene
        for b in bodies:
            b['obj'].pos = b['pos']

        t += h

In [ ]:
Place holder

In [ ]:
simulate()

In [ ]:
Place holder

In [ ]:
Place holder